In [ ]:
import xmlrpc.client
import pymssql

# Custom transport with a timeout
class TimeoutTransport(xmlrpc.client.Transport):
    def __init__(self, timeout=None):
        super().__init__()
        self.timeout = timeout

    def make_connection(self, host):
        connection = super().make_connection(host)
        connection.timeout = self.timeout
        return connection

# Odoo Connection Details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# SQL Server Connection Details
server_name = 'SARAH\\SQLEXPRESS'
database_name = 'Odoo_sql_database'
sql_user_name = 'SuperAdmin'
sql_password = 'SuperAdmin'

# Define the date range filter (1/1/2025 to 31/1/2025)
date_domain = [
    ('date_order', '>=', '2024-01-01'),
    ('date_order', '<=', '2025-03-10')
]

# Initialize batch size for SQL insert
batch_size = 100

def fetch_data_in_bulk(model, fields, domain, uid):
    """Fetch data from Odoo in bulk for a specific model and domain."""
    try:
        models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object', transport=TimeoutTransport(timeout=300))
        data = models.execute_kw(
            db_name,
            uid,
            password,
            model,
            'search_read',
            [domain],
            {'fields': fields}
        )
        return data
    except Exception as e:
        print(f"Error fetching data from Odoo for model {model}: {e}")
        return []

def process_and_insert():
    print("Starting data synchronization...")

    # Step 1: Authenticate with Odoo
    try:
        common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common', transport=TimeoutTransport(timeout=300))
        uid = common.authenticate(db_name, username, password, {})
        if not uid:
            print("Failed to authenticate. Check your credentials.")
            return
        print(f"Authenticated successfully. User ID: {uid}")
    except Exception as e:
        print(f"Authentication error: {e}")
        return

    # Step 2: Fetch orders
    print("Fetching POS orders...")
    orders = fetch_data_in_bulk('pos.order', ['name', 'state', 'payment_ids'], date_domain, uid)

    if not orders:
        print("No orders fetched. Exiting.")
        return
    print(f"Fetched {len(orders)} orders.")

    # Step 3: Prefetch all payments
    print("Fetching related payments...")
    all_payment_ids = [payment_id for order in orders for payment_id in order.get('payment_ids', [])]
    payments = fetch_data_in_bulk('pos.payment', ['id', 'display_name', 'payment_method_id'], [['id', 'in', all_payment_ids]], uid)

    # Step 4: Map payment methods for faster lookups
    payment_method_ids = list({payment.get('payment_method_id', [])[0] for payment in payments if payment.get('payment_method_id')})
    payment_methods = fetch_data_in_bulk('pos.payment.method', ['id', 'name'], [['id', 'in', payment_method_ids]], uid)
    payment_method_map = {method['id']: method.get('name') for method in payment_methods}

    # Step 5: Prepare and batch insert data into SQL Server
    try:
        conn = pymssql.connect(
            server=server_name,
            user=sql_user_name,
            password=sql_password,
            database=database_name,
            charset='utf8'
        )
        cursor = conn.cursor()

        # Create table if it doesn't exist
        cursor.execute(f"""
        IF NOT EXISTS (
            SELECT * FROM INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_NAME = 'payments'
        )
        BEGIN
            CREATE TABLE payments (
                id INT IDENTITY(1,1) PRIMARY KEY,
                order_ref NVARCHAR(255),
                status NVARCHAR(255),
                payment_display_name NVARCHAR(255),
                payment_method NVARCHAR(255)
            );
        END
        """)
        conn.commit()

        print("Inserting data into SQL Server...")
        batch_data = []

        for order in orders:
            order_ref = order.get('name')
            status = order.get('state')
            payment_ids = order.get('payment_ids', [])

            for payment_id in payment_ids:
                payment = next((p for p in payments if p['id'] == payment_id), None)
                if not payment:
                    continue

                payment_display_name = payment.get('display_name')
                payment_method_id = payment.get('payment_method_id', [])[0] if payment.get('payment_method_id') else None
                payment_method = payment_method_map.get(payment_method_id, None)

                # Add to batch
                batch_data.append((order_ref, status, payment_display_name, payment_method))

                # Insert in batches
                if len(batch_data) >= batch_size:
                    cursor.executemany(f"""
                    INSERT INTO payments (
                        order_ref, status, payment_display_name, payment_method
                    ) VALUES (%s, %s, %s, %s)
                    """, batch_data)
                    conn.commit()
                    batch_data = []

        # Final batch insert
        if batch_data:
            cursor.executemany(f"""
            INSERT INTO payments (
                order_ref, status, payment_display_name, payment_method
            ) VALUES (%s, %s, %s, %s)
            """, batch_data)
            conn.commit()

        print("All data inserted successfully.")

        # Close SQL connection
        cursor.close()
        conn.close()
    except Exception as e:
        print(f"Error during SQL operations: {e}")

# Run the function
process_and_insert()

Starting data synchronization...
Authenticated successfully. User ID: 112
Fetching POS orders...
Fetched 111570 orders.
Fetching related payments...
Inserting data into SQL Server...
